# Recommender Systems
Using Popularity Ranking, User-Based Collaborative Filtering and Item-Based Collaborative Filtering to recommend movies based on user or item inputs. Recommenders scored using various metrics on an offline train-test split

## Imports

In [1]:
%pip install lifelines

     -------------------------------------- 409.4/409.4 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 82.1/82.1 kB ? eta 0:00:00
     ---------------------------------------- 48.9/48.9 kB 2.4 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4049 sha256=21657802f2ea79d307b4cd5b97d2fe30777b3c1534777a0c61add6f3934f739b
  Stored in directory: c:\users\rockw\appdata\local\pip\cache\wheels\e4\30\15\a77aac970f4acc37b9bb9c7f56e73c7c85087f15ff0fb4720f
Successfully built autograd-gamma
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Importing Libraries

from movie_rec_utils import *

import pandas as pd
from ydata_profiling import ProfileReport
import math
import numpy as np
import matplotlib.pyplot as plt
import difflib

from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import r2_score, mean_absolute_error
from lifelines.utils import concordance_index

In [2]:
# Reading Data
links_df = pd.read_csv('../Data/links.csv')
movies_df = pd.read_csv('../Data/movies.csv')
ratings_df = pd.read_csv('../Data/ratings.csv')
tags_df =pd.read_csv('../Data/tags.csv')

## Data Exploration

In [ ]:
# Creating y-data profiling reports as html file
movies_profile = ProfileReport(movies_df, minimal=True).to_file('movies_report.html')
ratings_profile = ProfileReport(ratings_df, minimal=True).to_file('ratings_report.html')

# Recommenders

## Popularity Ranking

In [ ]:
# Creating Column to Track Review Count During Aggregations
ratings_df['rating_count'] = 1

### Average Ranking, Filtered with Review Minimum

### Laplace Inspired Data Manipulation

### Cumulative Rating

## Collaborative Filtering

### Item-Based Collaborative Filtering

In [3]:
pivoted_ratings = pd.pivot_table(ratings_df, 
                                 values='rating', 
                                 index='userId', 
                                 columns='movieId')

In [5]:
similar_movies = find_similar(title='Nausicaä of the valley of the wind',
                              ratings_df=ratings_df,
                              movies_df=movies_df,
                              pivoted_df=pivoted_ratings,
                              n=7,
                              shared_thresh=5,
                              total_thresh=10,
                              more_data = False)

similar_movies

c:\Users\rockw\anaconda3\lib\site-packages\numpy\lib\function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\rockw\anaconda3\lib\site-packages\numpy\lib\function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


0    Monty Python's And Now for Something Completel...
1                             Road to Perdition (2002)
2                                          Thor (2011)
3                      X-Men Origins: Wolverine (2009)
4                       Guardians of the Galaxy (2014)
5                            North by Northwest (1959)
6                                   Blue Velvet (1986)
Name: title, dtype: object

### User-Based Collaborative Filtering

## Evaluating Recommenders with Offline Methods

In [13]:
# Spliting Training and Testing Data
train, test = train_test_split(ratings_df[['userId', 'movieId', 'rating']], test_size=0.1, random_state=42)

# Creating Training Data Frame
useritem_train = create_train(ratings_df, train)

In [ ]:
# Computing Cosine Similarity
cos_sim_df = pd.DataFrame(cosine_similarity(useritem_train, useritem_train), 
                          index=useritem_train.index, 
                          columns=useritem_train.index)

In [ ]:
# Predicting/Estimating Ratings for Test Data Using Custom Function and Similarity Matrix
test['estimated_rating'] = test.apply(lambda row: estimate_rating(row['userId'], row['movieId']), axis=1)

In [ ]:
# Computing Performance Metrics
scores_df = score_est(test['rating'], test['estimated_rating'])
display(scores_df)